In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from statsmodels.tsa.statespace.varmax import VARMAX
import matplotlib.pyplot as plt

In [3]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
plt.rcParams["font.family"] = "Times New Roman"
plt.style.use('seaborn-whitegrid')
plt.style.use('seaborn-poster')
plt.style.use('seaborn-dark-palette')
plt.rcParams["mathtext.fontset"] = "cm"

In [4]:
data = pd.read_csv('../dataset/insect/ant/location_in_mm.csv')
data_chamber = pd.read_csv('../dataset/insect/ant/time_series_chamber.csv')
#print(data)

In [5]:
data_x = data[['location_x', 'ant_id', 'colony_id']]
data_y = data[['location_y', 'ant_id', 'colony_id']]
#print(data_x, data_y)

In [6]:
data_x = pd.DataFrame(np.unique(data.time), columns = ['time'])
data_y = pd.DataFrame(np.unique(data.time), columns = ['time'])

k = 1
for i in tqdm(range(len(np.unique(data.colony_id)))):
    for j in tqdm(range(len(np.unique(data[data['colony_id'] == i+1]['ant_id'])))):
        tmp = data[(data['colony_id'] == i+1) & (data['ant_id']== j)]['location_x']
        tmp1 = data[(data['colony_id'] == i+1) & (data['ant_id']== j)]['location_y']
        tmp.reset_index(drop = True, inplace = True)
        tmp1.reset_index(drop = True, inplace = True)
        data_x = pd.concat([data_x, tmp], axis = 1)
        data_x.rename({'location_x':'%i'%k}, axis = 'columns', inplace = True)
        data_y = pd.concat([data_y, tmp1], axis = 1)
        data_y.rename({'location_y':'%i'%k}, axis = 'columns', inplace = True)
        #print(time_list)
        k += 1

data_x.drop(columns = ['time'], inplace = True)
data_y.drop(columns = ['time'], inplace = True)

100%|██████████| 3/3 [00:04<00:00,  1.61s/it]


In [7]:
datax_corr = data_x.corr()
datay_corr = data_y.corr()
cpl_x = list()
cpl_y = list()

num_ant = 13

for j in range(len(datax_corr)):
    if (datax_corr.iloc[num_ant, j] > 0.6):
        cpl_x.append('%i'%j)
    if (datay_corr.iloc[num_ant, j] > 0.6):
        cpl_y.append('%i'%j)

ant_related = list()
for j in cpl_x:
    if (j in cpl_y):
        ant_related.append(j)
         
print(ant_related)

['13', '28']


In [8]:
data_df = pd.DataFrame(data_chamber['%i'%num_ant])
data_df.rename({'%i'%num_ant:'chamber'}, axis = 'columns', inplace = True)
for j in ant_related:
    data_df = pd.concat([data_df, data_x[j]], axis = 1)
    data_df.rename({j:'location_x_%s'%j}, axis = 'columns', inplace = True)
    data_df = pd.concat([data_df, data_y[j]], axis = 1)
    data_df.rename({j:'location_y_%s'%j}, axis = 'columns', inplace = True)
print(data_df)

      chamber  location_x_13  location_y_13  location_x_28  location_y_28
0           4          38.89         162.52          16.97          28.78
1           4          38.89         162.52          15.11          27.10
2           4          38.89         162.52          22.93          30.13
3           4          38.89         162.52          27.48          29.77
4           4          38.89         162.52          27.31          25.67
...       ...            ...            ...            ...            ...
2875        5           0.00           0.00          35.23         151.88
2876        5           0.00           0.00          35.23         151.88
2877        5           0.00           0.00          35.23         151.88
2878        5           0.00           0.00          33.77         153.15
2879        5           0.00           0.00          34.45         153.61

[2880 rows x 5 columns]


In [9]:
train = data_df[:int(0.5*(len(data_df)))]
valid = data_df[int(0.5*(len(data_df))):]

In [11]:
tmp = []
for p in tqdm(range(5)):
    for q in tqdm(range(5)):
        try:
            tmp.append([VARMAX(train, order = (p,q)).fit().bic, p, q])
        except:
            tmp.append([None, p, q])
tmp = pd.DataFrame(tmp,columns = ['bic', 'p', 'q'])
print(tmp)
tmp[tmp['bic'] == tmp['bic'].min()]

  0%|          | 0/5 [00:00<?, ?it/s]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

 40%|████      | 2/5 [00:20<00:30, 10.17s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

 60%|██████    | 3/5 [00:52<00:38, 19.35s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

 80%|████████  | 4/5 [01:45<00:31, 31.85s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarni


 80%|████████  | 4/5 [06:41<01:48, 108.33s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

  0%|          | 0/5 [00:00<?, ?it/s]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

 20%|██        | 1/5 [05:31<22:05, 331.47s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: E

             bic  p  q
0            NaN  0  0
1   50546.388364  0  1
2   58453.674686  0  2
3   54846.086287  0  3
4   55207.972408  0  4
5   29413.179349  1  0
6   28837.504166  1  1
7   28601.630616  1  2
8   28754.428700  1  3
9   28802.109197  1  4
10  28415.277955  2  0
11  28557.368454  2  1
12  28505.156860  2  2
13  28569.033622  2  3
14  28549.566974  2  4
15  28233.808396  3  0
16  28382.877919  3  1
17  28533.919607  3  2
18  28602.208580  3  3
19  28635.118986  3  4
20  28221.163784  4  0
21  28399.862440  4  1
22  28547.156117  4  2
23  28698.133584  4  3
24  28768.649490  4  4


,bic,p,q
20,28221.163784,4,0


In [13]:
print(tmp)

             bic  p  q
0            NaN  0  0
1   50546.388364  0  1
2   58453.674686  0  2
3   54846.086287  0  3
4   55207.972408  0  4
5   29413.179349  1  0
6   28837.504166  1  1
7   28601.630616  1  2
8   28754.428700  1  3
9   28802.109197  1  4
10  28415.277955  2  0
11  28557.368454  2  1
12  28505.156860  2  2
13  28569.033622  2  3
14  28549.566974  2  4
15  28233.808396  3  0
16  28382.877919  3  1
17  28533.919607  3  2
18  28602.208580  3  3
19  28635.118986  3  4
20  28221.163784  4  0
21  28399.862440  4  1
22  28547.156117  4  2
23  28698.133584  4  3
24  28768.649490  4  4


In [ ]:
prediction_x = list()
prediction_y = list()

for i in range(len(train)):
    prediction_x.append(train.iloc[i, 1])
    prediction_y.append(train.iloc[i, 2])

start_t = len(train)
for t_i in tqdm(range(len(valid))):
    current_t = t_i + start_t
    model = VARMAX(data_df[t_i:current_t], order = (3,0))
    fitted_model = model.fit()
    prediction = fitted_model.forecast().reset_index(drop=True) 
    #print(prediction)
    prediction_x.append(prediction['location_x_%i'%num_ant]) 
    prediction_y.append(prediction['location_y_%i'%num_ant]) 